In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms
from torchvision import datasets

batch_size = 4
learning_rate = 0.01
num_epochs = 5

transform = transforms.ToTensor()

# Data Loader
trainData = datasets.CIFAR10(root="./data",train=True,transform=transform,download=True)
trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True)

testData = datasets.CIFAR10(root="./data", train=False, transform=transform)
testLoader = DataLoader(testData, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [2]:
def conv(c_in, c_out, k_size, stride=1, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad, bias=False))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    layers.append(nn.ReLU())
    return nn.Sequential(*layers)

## 과제. VGG19 구현하기

![VGG19](https://qcloud.coding.net/u/vincentqin/p/blogResource/git/raw/master/CNN-Architectures/89475867.png)

In [3]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()

        self.layer1 = nn.Sequential(
            conv(3, 64, 3),
            conv(64, 64, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            conv(64, 128, 3),
            conv(128, 128, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            conv(128, 256, 3),
            conv(256, 256, 3),
            conv(256, 256, 3),
            conv(256, 256, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = nn.Sequential(
            conv(256, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer5 = nn.Sequential(
            conv(512, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.FC1 = nn.Sequential(
            nn.Linear(512, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.FC2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.FC3 = nn.Sequential(
            nn.Linear(4096, 10),
            nn.BatchNorm1d(10),
            nn.Softmax())

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)

        out = out.view(out.size(0), -1)

        out = self.FC1(out)
        out = self.FC2(out)
        out = self.FC3(out)

        return out

In [4]:
vgg19 = VGG19()

# Loss Function
Loss_function = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(vgg19.parameters(), lr=learning_rate)


In [ ]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainLoader):
        optimizer.zero_grad()
        outputs = vgg19(images)
        loss = Loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss %.4f'
                  % (epoch+1, num_epochs, i, len(trainLoader), loss.item()))

C:\Users\hyunju\Anaconda3\lib\site-packages\torch\nn\modules\container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Iter [0/12500] Loss 2.2938
Epoch [1/5], Iter [100/12500] Loss 2.3296
Epoch [1/5], Iter [200/12500] Loss 2.2860
Epoch [1/5], Iter [300/12500] Loss 2.3168
Epoch [1/5], Iter [400/12500] Loss 2.2095
Epoch [1/5], Iter [500/12500] Loss 2.3146
Epoch [1/5], Iter [600/12500] Loss 2.3021
Epoch [1/5], Iter [700/12500] Loss 2.2708
Epoch [1/5], Iter [800/12500] Loss 2.3000
Epoch [1/5], Iter [900/12500] Loss 2.2451
Epoch [1/5], Iter [1000/12500] Loss 2.3361
Epoch [1/5], Iter [1100/12500] Loss 2.3177
Epoch [1/5], Iter [1200/12500] Loss 2.2854
Epoch [1/5], Iter [1300/12500] Loss 2.3063
Epoch [1/5], Iter [1400/12500] Loss 2.3552
Epoch [1/5], Iter [1500/12500] Loss 2.2813
Epoch [1/5], Iter [1600/12500] Loss 2.2879
Epoch [1/5], Iter [1700/12500] Loss 2.2341
Epoch [1/5], Iter [1800/12500] Loss 2.1801
Epoch [1/5], Iter [1900/12500] Loss 2.3412
Epoch [1/5], Iter [2000/12500] Loss 2.2932
Epoch [1/5], Iter [2100/12500] Loss 2.2840
Epoch [1/5], Iter [2200/12500] Loss 2.2203
Epoch [1/5], Iter [2300

In [ ]:
# Test the model
vgg19.eval()
correct = 0
total = 0

for images, labels in testLoader:
    images = Variable(images).cuda()
    outputs = vgg19(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy : %d %%' % (100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_nn.ckpt')